In [1]:
# General
import pandas as pd
import numpy as np
import os
import glob

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

# Modelling
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
## Linear regression
from sklearn.linear_model import LinearRegression
## SVR
from sklearn.svm import LinearSVR, SVR
## Neural networks
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
## LGBM
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback

In [2]:
# Get rid of annoying LGBM messages
import warnings
warnings.filterwarnings("ignore", message="categorical_column in param dict is overridden.")
warnings.filterwarnings("ignore", message='Overriding the parameters from Reference Dataset.')
warnings.filterwarnings("ignore", message='The reported value is ignored because this*')
warnings.filterwarnings("ignore", message='Found `n_estimators` in params. Will use it*')
warnings.filterwarnings("ignore", message='The distribution is specified by*')

# Hide optuna logging too
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Pre-processing

In [3]:
# Get files to read in
gcse_files = glob.glob("../fake_data/synthetic_*_gcse_20[1-2][0, 8-9].csv")
npd_files = glob.glob("../fake_data/synthetic_npd_ks4_student_20[1-2][0, 8-9].csv")

## Exam Data

In [4]:
def process_grades(data = pd.DataFrame, grade_col = str):
    
    # Drop rows with missing grades
    data = data.dropna(subset = grade_col)
    # Convert U grade to 0
    data.loc[data[grade_col] == "U", grade_col] = "0"
    # Convert grades to numeric from string format
    data = data[data[grade_col].isin([str(x) for x in (range(0, 10))])]
    data[grade_col] = data[grade_col].astype(float)
    return data

In [5]:
def process_gcse_data(df = pd.DataFrame):
    
    """
    Takes raw GCSE exam data (2017-2020 files), filters it
    appropriately and processes it. 
    Returns a DataFrame with a reduced number of columns.
    Full steps taken can be seen in code commenting or in
    Methodology section of capstone.
    --------------------------------------------------
    df = DataFrame of raw GCSE data
    """
    
    # Copy to prevent in-place changes
    data = df.copy()
    
    # Make cols lowercase
    data.columns = [x.lower() for x in data.columns]
    
    # Reformat examseries to year col
    data["year"] = data.examseries.apply(lambda x: x.split()[1])
    
    # Remove candidates who were not 16 on 31st August
    data = data.query("yearendage == 16")
    # Remove private candidates
    data = data.query("privatecandidate == False")
    # Commented out below since all True in synthetic data
    # Remove partial absentees
#     data = data.query("partialabsence == False")
    # Remove candidates without prior attainment or that weren't matched in NPD
    data = data.dropna(subset = ["normalisedks2score", "npdmatchround"])
    
    # Remove candidates with 0 prior attainment (errors in data)
    data = data[data.normalisedks2score > 0]
    
    # Remove non-reformed GCSEs
    data = data[data.reformphase.isin(['Ofqual-regulated Phase 1 reformed GCSE FC',
                                       'Ofqual-regulated Phase 2 reformed GCSE FC'])]
    # Recode tier into foundation or not foundation
    data.loc[data.tier != "F", "tier"] = "Not F"
    
    # Process grade column inplace
    data = process_grades(data, grade_col = "grade")
    
    # Standardise the KS2 prior attainment to between 0 and 1
    scaler = MinMaxScaler()
    data.normalisedks2score = scaler.fit_transform(data[['normalisedks2score']])
    
    # Get candidates who took at least 8 GCSEs
    grouped = data.groupby("uidp").count()
    at_least_8 = set(grouped[grouped.examseries >= 8].index.to_list())
    # Get candidates who took English and Maths
    eng_math = set(data[data.jcqtitle.isin(["Mathematics", "English language"])].uidp)
    # Get candidates who took English and Maths and >= 8 GCSEs
    filtered_ids = at_least_8 & eng_math
    # Beware that since this is simulated data, it's wrong
    filtered = data[data.uidp.isin(filtered_ids)]
    
    # Select cols needed for modelling and dropnas
    gcse_cols = ["uidp", "year", "jcqtitle", "tier", "centretypedesc",
                 "normalisedks2score", "grade", "centreassessmentgrade"]
    filtered = filtered[gcse_cols]

    return filtered

In [6]:
# Load and process all the GCSE exam data
gcse_data = pd.DataFrame()
# Iterate through files
for file in gcse_files:
    # Perform filtering/pre-processing
    year_df = process_gcse_data(pd.read_csv(file))
    # Process the CAG column too
    if "2020" in file:
        year_df = process_grades(year_df, "centreassessmentgrade")
    # Create dummy value for other years
    else:
        year_df.centreassessmentgrade = np.NaN
        
    # Merge with other years
    gcse_data = pd.concat([gcse_data, year_df])
    # Delete var to save memory
    del year_df
# Reset index
gcse_data = gcse_data.reset_index(drop = True)

## NPD Data

In [7]:
def process_npd(data = pd.DataFrame):
    
    """
    Takes raw NPD data (2017-2020 files), filters it
    appropriately and processes it. 
    Returns a DataFrame with a reduced number of columns.
    Full steps taken can be seen in code commenting or in
    Methodology section of capstone.
    --------------------------------------------------
    df = DataFrame of raw NPD data
    """    
    
    # Copy to prevent inplace changes
    df = data.copy()
    # Make cols lowercase
    df.columns = [x.lower() for x in df.columns]
    # Select the columns that are common across files
    npd_cols = ["uidp", "ks4_ealgrp_ptq_ee", "ks4_gender"]
    # Get the bases for the columns that change in suffix in each file
    col_bases = ["ethnicgroupmajor", "fsmeligible", "senprovisionmajor"]
    # Get the suffix part that changes
    year_ending = int(file[-6:-4])
    # Dynamically select those cols with changing suffixes
    npd_cols.extend([col_base + f"_spr{year_ending}" for col_base in col_bases])
    # Also add in most recent IDACI score
    npd_cols.append(sorted([x for x in df.columns if "idaciscore" in x])[-1])
    
    # Select the needed columns
    df = df[npd_cols]
    # Add in year col
    df["year"] = f"20{year_ending}"
    # Rename columns
    clean_cols = ["uidp", "eal", "gender", "ethnicity",
              "fsm", "sen", "idaci", "year"]
    df.columns = clean_cols
    
    return df

In [8]:
col_dict = dict()
for file in npd_files:
    col_dict[file[-8:-4]] = pd.read_csv(file).columns

In [9]:
# set(col_dict["2020"]) & set(col_dict["2019"]) & set(col_dict["2018"])

In [10]:
# set(col_dict["2020"]) - set(col_dict["2019"])

In [11]:
# Create df to store each year's data in
npd_data = pd.DataFrame()

# Iterate through files
for file in npd_files:
    # Load data
    df = pd.read_csv(file)
    # Process the NPD data
    df = process_npd(df)
    # Combine into dataframe
    npd_data = pd.concat([npd_data, df])

# Joining

In [12]:
def recode_cols(data = pd.DataFrame):
    """
    Takes processed merged GCSE exam and NPD data (2017-2020 files),
    filters it appropriately and processes it. 
    It recodes several columns into fewer numbers of categories
    to make modelling easier.
    Returns a DataFrame with a reduced number of columns.
    Full steps taken can be seen in code commenting or in
    Methodology section of capstone.
    --------------------------------------------------
    df = DataFrame of merged NPD/GCSE data
    """
    
    # Copy to prevent inplace changes
    df = data.copy()
    # Filter EAL to remove NAs or unclassifieds
    df = df[df.eal.isin([1,2])]
    # Filter ethnicity to remove unclassifieds/NaNs
    df = df[df.ethnicity.isin(["AOEG", "ASIA", "BLAC", "CHIN",
                          "MIXD", "WHIT"])]
    # Filter and recode SEN to remove unclassifieds and make SEN/not SEN
    df = df[df.sen.isin(["1_NON", "2_SNS", "3_SS"])]
    df.loc[df.sen != "1_NON", "sen"] = "SEN"
    df.loc[df.sen == "1_NON", "sen"] = "No SEN"
    
    # Drop remaining NaNs from FSM and IDACI cols
    df = df.dropna(subset = ["fsm", "idaci"])
    
    return df

In [13]:
# Inner join exam data with NPD data
merged = npd_data.merge(gcse_data, on = ["uidp", "year"],
                       how = "inner")

# Recode columns and filter further
df = recode_cols(merged)

# Drop now unnecesary UIDP and year cols
df = df.drop(columns = ["year", "uidp"])

# Convert categorical cols to numerics
categorical_cols = ["eal", "gender", "ethnicity", "fsm",
               "sen", "jcqtitle", "tier", "centretypedesc"]
# Fit encoder on categorical cols
encoder = OrdinalEncoder()
encoder.fit(df[categorical_cols])

# Create a mapping for reference later
mapping = {k:v for k, v in zip(categorical_cols, encoder.categories_)}

# Convert categoricals to numerics
df[categorical_cols] = encoder.transform(df[categorical_cols])

# Split into treatment and control
treatment = df[~df.centreassessmentgrade.isna()]
control = df[df.centreassessmentgrade.isna()]
# Old code, maybe useful for LGBM
# df[categorical_cols] = df[categorical_cols].apply(pd.Categorical)

In [14]:
# Split into labels and features
X = np.array(control.iloc[:, :10], dtype = "float32")
y = np.array(control.grade, dtype = "float32")

# Split into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                   shuffle = True,
                                                   random_state = 42)

# Quick EDA / Check

In [15]:
from pandas_profiling import ProfileReport

In [16]:
report = ProfileReport(df, title = "eda_check")
report.to_file("eda_check.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Modelling

In [17]:
# Create dataframe to store model results in
all_results = pd.DataFrame()

In [18]:
def evaluate_model(X_train, X_test,
                  y_train, y_test,
                  model, model_name):
    
    """
    Function to evaluate a model in terms of
    train and test RMSE.
    Returns a dataframe of model name and RMSEs.
    --------------------------------------------------
    X_train = np.array of X data, used to generate train RMSE
    X_test = np.array of X data, used to generate test RMSE
    y_train = np.array of y data, used to generate train RMSE
    y_test = np.array of y data, used to generate test RMSE
    model = fitted model instance to use with model.predict
    model_name = str, name to save the model under
    """
    # Generate predictions
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)
    # Evaluate model
    train_rmse = mean_squared_error(y_train, train_preds, squared = False)
    test_rmse = mean_squared_error(y_test, test_preds, squared = False)

    # Store results
    results = pd.DataFrame({"model": model_name,
                            "train_rmse": train_rmse,
                            "test_rmse": test_rmse,
                 }, index = [0])
    
    return results

## Linear Model

In [19]:
# Create linear model
model = LinearRegression()
model.fit(X_train, y_train)
# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "linear")
# Store results
all_results = pd.concat([all_results, results])

## Neural Network

In [20]:
def build_mlp(X_data,
              layer_1_units = 64,
              layer_2_units = 64,
              batch_normalization = False,
              loss = "mse",
              optimizer = "adam",
              metrics = ["mse"]):
    """
    Function to create artificial neural network. Dense layer
    units can be specified, as can the use of batch normalization
    in between the dense layers (this provides mild regularisation)
    and may speed up training.
    Returns a compiled Keras model.
    --------------------------------------------------
    X_data = np.array of X data, used to give input shape to model
    layer_1_units = int, number of neurons in 1st hidden layer
    layer_2_units = int, number of neurons in 2nd hidden layer
    batch_normalization = bool, batch normalize between hidden layers 
    if true
    loss = str, name of loss function to use
    optimizer = str or keras.Optimzer object, optimizer to use
    metrics = list of strings, evaluation metrics to use
    """
    # Build model
    model = Sequential(name = "MLP")
    # 1st Dense layer
    model.add(Dense(units = layer_1_units, activation = "relu", input_shape = (X_data.shape[1], ),
                   kernel_initializer = "he_normal"))
    
    # Add batch normalization if desired
    if batch_normalization:
        model.add(BatchNormalization())
    
    # 2nd Dense layer
    model.add(Dense(units = layer_2_units, activation = "relu",
                   kernel_initializer = "he_normal"))
    # Output layer
    model.add(Dense(units = 1, activation = "linear",
                   kernel_initializer = "he_normal"))
    # Compile model
    model.compile(**compile_hp)
    
    return model

In [21]:
# Hyperparams used during modelling
# Compilation hyperparams
compile_hp = dict()
compile_hp["loss"] = "mse"
compile_hp["optimizer"] = optimizers.Adam(learning_rate = 0.001)
compile_hp["metrics"] = ["mse"]

# Fitting hyperparams
fit_hp = dict()
fit_hp["batch_size"] = 32
fit_hp["epochs"] = 200
fit_hp["validation_split"] = 0.2
# Create callback to select the best model
fit_hp["callbacks"] = EarlyStopping(monitor = "val_loss",
                                         mode = "min",
                                         restore_best_weights = True,
                                         patience = 25)

# Eliminate verbose to have a neater notebook 
fit_hp["verbose"] = 2

### NN 1

In [22]:
# Select number of hidden units
layer_1_units = 64
layer_2_units = 64
# Select whether to batch normalize
batch_normalization = True

# Build and compile model
model = build_mlp(X_train,
                  layer_1_units = layer_1_units,
                  layer_2_units = layer_2_units,
                  batch_normalization = batch_normalization,
                  **compile_hp)
# Fit model
history = model.fit(X_train, y_train, **fit_hp)

# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "neural_network-64_64_init_bn")
# Store results
all_results = pd.concat([all_results, results])

Metal device set to: Apple M1


2022-07-23 17:19:54.296737: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-23 17:19:54.296851: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2022-07-23 17:19:54.669986: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-23 17:19:54.890119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 - 1s - loss: 35.4301 - mse: 35.4301 - val_loss: 105.5467 - val_mse: 105.5467 - 526ms/epoch - 175ms/step
Epoch 2/200
3/3 - 0s - loss: 31.6174 - mse: 31.6174 - val_loss: 73.1889 - val_mse: 73.1889 - 31ms/epoch - 10ms/step
Epoch 3/200
3/3 - 0s - loss: 27.8028 - mse: 27.8028 - val_loss: 50.8299 - val_mse: 50.8299 - 31ms/epoch - 10ms/step
Epoch 4/200
3/3 - 0s - loss: 24.8443 - mse: 24.8443 - val_loss: 35.7308 - val_mse: 35.7308 - 32ms/epoch - 11ms/step
Epoch 5/200
3/3 - 0s - loss: 21.7518 - mse: 21.7518 - val_loss: 25.5623 - val_mse: 25.5623 - 31ms/epoch - 10ms/step
Epoch 6/200


2022-07-23 17:19:55.161595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 - 0s - loss: 19.2080 - mse: 19.2080 - val_loss: 18.7571 - val_mse: 18.7571 - 31ms/epoch - 10ms/step
Epoch 7/200
3/3 - 0s - loss: 17.3442 - mse: 17.3442 - val_loss: 14.3797 - val_mse: 14.3797 - 31ms/epoch - 10ms/step
Epoch 8/200
3/3 - 0s - loss: 14.9395 - mse: 14.9395 - val_loss: 11.7763 - val_mse: 11.7762 - 36ms/epoch - 12ms/step
Epoch 9/200
3/3 - 0s - loss: 13.3005 - mse: 13.3005 - val_loss: 10.6008 - val_mse: 10.6008 - 31ms/epoch - 10ms/step
Epoch 10/200
3/3 - 0s - loss: 11.9022 - mse: 11.9022 - val_loss: 10.2842 - val_mse: 10.2842 - 31ms/epoch - 10ms/step
Epoch 11/200
3/3 - 0s - loss: 10.7360 - mse: 10.7360 - val_loss: 10.5100 - val_mse: 10.5100 - 28ms/epoch - 9ms/step
Epoch 12/200
3/3 - 0s - loss: 9.3650 - mse: 9.3650 - val_loss: 11.0057 - val_mse: 11.0057 - 28ms/epoch - 9ms/step
Epoch 13/200
3/3 - 0s - loss: 8.2038 - mse: 8.2038 - val_loss: 11.6204 - val_mse: 11.6204 - 30ms/epoch - 10ms/step
Epoch 14/200
3/3 - 0s - loss: 8.0011 - mse: 8.0011 - val_loss: 12.2988 - val_mse: 12.2

Epoch 79/200
3/3 - 0s - loss: 2.6666 - mse: 2.6666 - val_loss: 5.8638 - val_mse: 5.8638 - 30ms/epoch - 10ms/step
Epoch 80/200
3/3 - 0s - loss: 2.9854 - mse: 2.9854 - val_loss: 5.8453 - val_mse: 5.8453 - 31ms/epoch - 10ms/step
Epoch 81/200
3/3 - 0s - loss: 2.8774 - mse: 2.8774 - val_loss: 5.8236 - val_mse: 5.8236 - 33ms/epoch - 11ms/step
Epoch 82/200
3/3 - 0s - loss: 2.8302 - mse: 2.8302 - val_loss: 5.8345 - val_mse: 5.8345 - 31ms/epoch - 10ms/step
Epoch 83/200
3/3 - 0s - loss: 2.7344 - mse: 2.7344 - val_loss: 5.8675 - val_mse: 5.8675 - 28ms/epoch - 9ms/step
Epoch 84/200
3/3 - 0s - loss: 2.6378 - mse: 2.6378 - val_loss: 5.9082 - val_mse: 5.9082 - 30ms/epoch - 10ms/step
Epoch 85/200
3/3 - 0s - loss: 2.3327 - mse: 2.3327 - val_loss: 5.9474 - val_mse: 5.9474 - 29ms/epoch - 10ms/step
Epoch 86/200
3/3 - 0s - loss: 2.8361 - mse: 2.8361 - val_loss: 5.9917 - val_mse: 5.9917 - 29ms/epoch - 10ms/step
Epoch 87/200
3/3 - 0s - loss: 2.8071 - mse: 2.8071 - val_loss: 6.0313 - val_mse: 6.0313 - 30ms/ep

2022-07-23 17:19:58.634692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### NN 2

In [23]:
# Select number of hidden units
layer_1_units = 64
layer_2_units = 128
# Select whether to batch normalize
batch_normalization = False

# Build and compile model
model = build_mlp(X_train,
                  layer_1_units = layer_1_units,
                  layer_2_units = layer_2_units,
                  batch_normalization = batch_normalization,
                  **compile_hp)
# Fit model
history = model.fit(X_train, y_train, **fit_hp)

# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "neural_network-64_128")
# Store results
all_results = pd.concat([all_results, results])

Epoch 1/200
3/3 - 0s - loss: 14.6642 - mse: 14.6642 - val_loss: 18.0414 - val_mse: 18.0414 - 289ms/epoch - 96ms/step
Epoch 2/200
3/3 - 0s - loss: 14.8874 - mse: 14.8874 - val_loss: 8.2808 - val_mse: 8.2808 - 27ms/epoch - 9ms/step
Epoch 3/200


2022-07-23 17:19:58.882918: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-23 17:19:59.012326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 - 0s - loss: 6.3675 - mse: 6.3675 - val_loss: 11.7070 - val_mse: 11.7070 - 26ms/epoch - 9ms/step
Epoch 4/200
3/3 - 0s - loss: 6.1703 - mse: 6.1703 - val_loss: 5.9830 - val_mse: 5.9830 - 27ms/epoch - 9ms/step
Epoch 5/200
3/3 - 0s - loss: 5.6631 - mse: 5.6631 - val_loss: 6.0599 - val_mse: 6.0599 - 25ms/epoch - 8ms/step
Epoch 6/200
3/3 - 0s - loss: 4.5408 - mse: 4.5408 - val_loss: 7.9040 - val_mse: 7.9040 - 27ms/epoch - 9ms/step
Epoch 7/200
3/3 - 0s - loss: 4.5045 - mse: 4.5045 - val_loss: 5.8169 - val_mse: 5.8169 - 30ms/epoch - 10ms/step
Epoch 8/200
3/3 - 0s - loss: 4.8580 - mse: 4.8580 - val_loss: 5.9140 - val_mse: 5.9140 - 27ms/epoch - 9ms/step
Epoch 9/200
3/3 - 0s - loss: 4.3725 - mse: 4.3725 - val_loss: 7.8202 - val_mse: 7.8202 - 25ms/epoch - 8ms/step
Epoch 10/200
3/3 - 0s - loss: 4.4612 - mse: 4.4612 - val_loss: 6.3789 - val_mse: 6.3789 - 26ms/epoch - 9ms/step
Epoch 11/200
3/3 - 0s - loss: 3.6741 - mse: 3.6741 - val_loss: 6.0085 - val_mse: 6.0085 - 27ms/epoch - 9ms/step
Epoch 12

2022-07-23 17:19:59.902471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### NN 3

In [24]:
# Select number of hidden units
layer_1_units = 128
layer_2_units = 128
# Select whether to batch normalize
batch_normalization = False

# Build and compile model
model = build_mlp(X_train,
                  layer_1_units = layer_1_units,
                  layer_2_units = layer_2_units,
                  batch_normalization = batch_normalization,
                  **compile_hp)
# Fit model
history = model.fit(X_train, y_train, **fit_hp)

# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "neural_network-128_128")
# Store results
all_results = pd.concat([all_results, results])

Epoch 1/200
3/3 - 0s - loss: 20.1776 - mse: 20.1776 - val_loss: 27.4567 - val_mse: 27.4567 - 282ms/epoch - 94ms/step
Epoch 2/200
3/3 - 0s - loss: 21.1520 - mse: 21.1520 - val_loss: 15.2873 - val_mse: 15.2873 - 27ms/epoch - 9ms/step
Epoch 3/200


2022-07-23 17:20:00.114985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-23 17:20:00.243560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 - 0s - loss: 14.4204 - mse: 14.4204 - val_loss: 8.9692 - val_mse: 8.9692 - 28ms/epoch - 9ms/step
Epoch 4/200
3/3 - 0s - loss: 8.3185 - mse: 8.3185 - val_loss: 9.9003 - val_mse: 9.9003 - 24ms/epoch - 8ms/step
Epoch 5/200
3/3 - 0s - loss: 8.2991 - mse: 8.2991 - val_loss: 7.3524 - val_mse: 7.3524 - 26ms/epoch - 9ms/step
Epoch 6/200
3/3 - 0s - loss: 6.7945 - mse: 6.7945 - val_loss: 9.3715 - val_mse: 9.3715 - 24ms/epoch - 8ms/step
Epoch 7/200
3/3 - 0s - loss: 5.1064 - mse: 5.1064 - val_loss: 5.7419 - val_mse: 5.7419 - 26ms/epoch - 9ms/step
Epoch 8/200
3/3 - 0s - loss: 5.9638 - mse: 5.9638 - val_loss: 5.8756 - val_mse: 5.8756 - 24ms/epoch - 8ms/step
Epoch 9/200
3/3 - 0s - loss: 3.9530 - mse: 3.9530 - val_loss: 8.4094 - val_mse: 8.4094 - 24ms/epoch - 8ms/step
Epoch 10/200
3/3 - 0s - loss: 4.6990 - mse: 4.6990 - val_loss: 6.8237 - val_mse: 6.8237 - 26ms/epoch - 9ms/step
Epoch 11/200
3/3 - 0s - loss: 3.8229 - mse: 3.8229 - val_loss: 6.2162 - val_mse: 6.2162 - 24ms/epoch - 8ms/step
Epoch 12/

2022-07-23 17:20:01.177584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### NN 4

In [25]:
# Select number of hidden units
layer_1_units = 32
layer_2_units = 32
# Select whether to batch normalize
batch_normalization = False,

# Build and compile model
model = build_mlp(X_train,
                  layer_1_units = layer_1_units,
                  layer_2_units = layer_2_units,
                  batch_normalization = batch_normalization,
                  **compile_hp)
# Fit model
history = model.fit(X_train, y_train, **fit_hp)

# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "neural_network-32_32")
# Store results
all_results = pd.concat([all_results, results])

Epoch 1/200


2022-07-23 17:20:01.461789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-23 17:20:01.655970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 - 0s - loss: 21.8550 - mse: 21.8550 - val_loss: 11.3112 - val_mse: 11.3112 - 413ms/epoch - 138ms/step
Epoch 2/200
3/3 - 0s - loss: 18.9942 - mse: 18.9942 - val_loss: 17.8961 - val_mse: 17.8961 - 29ms/epoch - 10ms/step
Epoch 3/200
3/3 - 0s - loss: 15.2380 - mse: 15.2380 - val_loss: 32.3568 - val_mse: 32.3568 - 28ms/epoch - 9ms/step
Epoch 4/200
3/3 - 0s - loss: 12.8583 - mse: 12.8583 - val_loss: 49.4785 - val_mse: 49.4785 - 28ms/epoch - 9ms/step
Epoch 5/200
3/3 - 0s - loss: 10.2925 - mse: 10.2925 - val_loss: 59.5824 - val_mse: 59.5824 - 28ms/epoch - 9ms/step
Epoch 6/200
3/3 - 0s - loss: 8.6520 - mse: 8.6520 - val_loss: 62.2779 - val_mse: 62.2779 - 28ms/epoch - 9ms/step
Epoch 7/200
3/3 - 0s - loss: 7.9952 - mse: 7.9952 - val_loss: 54.7621 - val_mse: 54.7621 - 29ms/epoch - 10ms/step
Epoch 8/200
3/3 - 0s - loss: 6.5431 - mse: 6.5431 - val_loss: 41.8957 - val_mse: 41.8957 - 28ms/epoch - 9ms/step
Epoch 9/200
3/3 - 0s - loss: 5.5656 - mse: 5.5656 - val_loss: 30.3596 - val_mse: 30.3596 - 28

Epoch 74/200
3/3 - 0s - loss: 2.3627 - mse: 2.3627 - val_loss: 6.5438 - val_mse: 6.5438 - 28ms/epoch - 9ms/step
Epoch 75/200
3/3 - 0s - loss: 2.3729 - mse: 2.3729 - val_loss: 6.5228 - val_mse: 6.5228 - 28ms/epoch - 9ms/step
Epoch 76/200
3/3 - 0s - loss: 2.2508 - mse: 2.2508 - val_loss: 6.5294 - val_mse: 6.5294 - 28ms/epoch - 9ms/step
Epoch 77/200
3/3 - 0s - loss: 2.4313 - mse: 2.4313 - val_loss: 6.5436 - val_mse: 6.5436 - 28ms/epoch - 9ms/step
Epoch 78/200
3/3 - 0s - loss: 2.3667 - mse: 2.3667 - val_loss: 6.5413 - val_mse: 6.5413 - 28ms/epoch - 9ms/step
Epoch 79/200
3/3 - 0s - loss: 2.3822 - mse: 2.3822 - val_loss: 6.4721 - val_mse: 6.4721 - 28ms/epoch - 9ms/step
Epoch 80/200
3/3 - 0s - loss: 2.3036 - mse: 2.3036 - val_loss: 6.4798 - val_mse: 6.4798 - 28ms/epoch - 9ms/step
Epoch 81/200
3/3 - 0s - loss: 2.2780 - mse: 2.2780 - val_loss: 6.4866 - val_mse: 6.4866 - 28ms/epoch - 9ms/step
Epoch 82/200
3/3 - 0s - loss: 2.3489 - mse: 2.3489 - val_loss: 6.5255 - val_mse: 6.5255 - 28ms/epoch - 9

Epoch 147/200
3/3 - 0s - loss: 2.0627 - mse: 2.0627 - val_loss: 6.4408 - val_mse: 6.4408 - 28ms/epoch - 9ms/step
Epoch 148/200
3/3 - 0s - loss: 1.8104 - mse: 1.8104 - val_loss: 6.3905 - val_mse: 6.3905 - 28ms/epoch - 9ms/step
Epoch 149/200
3/3 - 0s - loss: 1.9432 - mse: 1.9432 - val_loss: 6.3619 - val_mse: 6.3619 - 28ms/epoch - 9ms/step
Epoch 150/200
3/3 - 0s - loss: 2.1959 - mse: 2.1959 - val_loss: 6.4055 - val_mse: 6.4055 - 28ms/epoch - 9ms/step
Epoch 151/200
3/3 - 0s - loss: 2.0215 - mse: 2.0215 - val_loss: 6.4596 - val_mse: 6.4596 - 31ms/epoch - 10ms/step


2022-07-23 17:20:06.312688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [26]:
all_results.sort_values("test_rmse")

,model,train_rmse,test_rmse
0,linear,1.969422,1.888512
0,neural_network-32_32,1.690016,2.073730
0,neural_network-64_128,2.119378,2.127424
0,neural_network-128_128,2.424878,2.326648
0,neural_network-64_64_init_bn,1.849187,2.391446


## LGBM

In [27]:
# Params to compile model with
fixed_params = {
        'objective': 'regression',
        'metric': "rmse",  
        'verbosity': -1,
}

In [28]:
def objective(trial, X, y):
    """
    Wrapper function to work with Optuna trial objects, 
    enabling Hyperband hyperparameter search.
    """   
    # Suggest hyperparams to test using Optuna trial object.
    param = {**fixed_params,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 3000, step = 20),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.99, step = 0.05),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.99, step = 0.05),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        "n_estimators": trial.suggest_int("n_estimators", 200, 5000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 2000, step=5),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 10),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
    }
    
    # Create cv object
    cv = StratifiedKFold(n_splits = 5, shuffle = True)
    # Make empty array to store cv RMSE scores in
    cv_scores = np.empty(5)
    
    # Split into K train and validation sets and iterate through them
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        # Split into training and validation CV sets
        X_train_cv, X_test_cv = X[train_idx], X[test_idx]
        y_train_cv, y_test_cv = y[train_idx], y[test_idx]

        # Convert data to proper LGBM format
        train_data = lgb.Dataset(X_train_cv, label = y_train_cv,
                                 categorical_feature = [0,1,2,3,4,6,7,8])
        val_data = lgb.Dataset(X_test_cv, label = y_test_cv, 
                               categorical_feature = [0,1,2,3,4,6,7,8],
                              reference = train_data)
        
        # Make callbacks to prevent trialling hyperparams that are obviously bad
        callbacks = [
            LightGBMPruningCallback(trial, metric = "rmse"),
                     # Callback to reduce model validation performance messages
                    lgb.log_evaluation(period = 100),
                     # Early stoppping to prevent overfitting training data
                    lgb.early_stopping(100)]

        # Training the model
        model = lgb.train(params = param,  train_set = train_data,
                          valid_sets = val_data,   
                          callbacks = callbacks,
                         )
    
        # Get predictions
        preds = model.predict(X_test_cv)
        # Calculate RMSE
        cv_scores[idx] = mean_squared_error(y_test_cv, preds, squared = False)

    return np.mean(cv_scores)

In [29]:
%%capture my_study
# Above line magic hides lengthy output, but stores into first_round if you want to look

# Create Optuna study to do CV hyperparameter search
study = optuna.create_study(direction = "minimize", # minimizing RMSE
                            study_name = "LGBM Classifier",
                           pruner = optuna.pruners.HyperbandPruner())
func = lambda trial: objective(trial, X = X_train, y = y_train)
study.optimize(func, n_trials = 200)

In [30]:
# Run best model and evaluate
# Convert data to proper LGBM format
train_data = lgb.Dataset(X_train, label = y_train,
                         categorical_feature = [0,1,2,3,4,6,7,8])

# Callback to reduce model messages
callbacks = [lgb.log_evaluation(period = 100)]

# Training the model using the best params identified in study
model = lgb.train(params = {**fixed_params, **study.best_params},
                  train_set = train_data, 
                  callbacks = callbacks,
                 )

# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         model, "lgbm")
# Store results
all_results = pd.concat([all_results, results])

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=69 will be ignored. Current value: min_data_in_leaf=5


/Users/louis/miniforge3/envs/skywalker/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


## Support Vector Regression

### RBF SVR

In [31]:
# Create model
svr = SVR()
# Fit
svr.fit(X_train, y_train)
# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         svr, "svm_rbf")
# Store results
all_results = pd.concat([all_results, results])

### LinearSVR

In [32]:
# Create model
epsilon = 0.499
svr = LinearSVR(epsilon = epsilon)
# Fit
svr.fit(X_train, y_train)
# Evaluate model, getting test and train RMSE
results = evaluate_model(X_train, X_test,
                         y_train, y_test,
                         svr, f"svm_linear-{epsilon}")
# Store results
all_results = pd.concat([all_results, results])
all_results.sort_values("test_rmse")

/Users/louis/miniforge3/envs/skywalker/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,model,train_rmse,test_rmse
0,svm_rbf,2.044915,1.732313
0,lgbm,1.915254,1.803835
0,linear,1.969422,1.888512
0,neural_network-32_32,1.690016,2.073730
0,svm_linear-0.499,2.054832,2.118775
0,neural_network-64_128,2.119378,2.127424
0,neural_network-128_128,2.424878,2.326648
0,neural_network-64_64_init_bn,1.849187,2.391446
